In [1]:
import numpy as np
import pandas as pd
import math

In [2]:
DATE = '2020_04_05'

data = pd.read_csv(f'countermeasures_features_{DATE}.csv').drop(columns='Unnamed: 0').fillna(0)

In [3]:


REPORTCOLS = {'Widespread mask usage (over 75%)':('Mask wearing',lambda x: x>=75),
              'Confirmed case isolation':('Symptomatic isolation - targeted',lambda x: x>=1),
              'Contact isolation':('Asymptomatic isolation - targeted',lambda x: x>=1),
              'Gatherings limited to 10 or less':('Gatherings banned',lambda x: (x>1) and (x<=10)),
              'Gatherings limited to 100':('Gatherings banned',lambda x: (x>10) and (x<=100)),
              'Gatherings limited to 500':('Gatherings banned',lambda x: (x>100) and (x<=500)),
              'Some businesses suspended':('Activity cancellation',lambda x:(x>=1) and (x<=2)),
              'Many businesses suspended':('Nonessential business suspension',lambda x: x>=1),
              'Schools and universities closed':('School closure',lambda x: x>=1),
              'Social distancing advised':('Public interaction and hygiene',lambda x: (x>=1) and (x<3)),
              'Staying home advised':('Public interaction and hygiene',lambda x: x>=3),
              '1 - 10 000 tests':('Testing',lambda x: (x>=1) and (x<=10000)),
              '10 000 - 100 000 tests':('Testing',lambda x: (x>10000) and (x<=1e5)),
              '100 000+ tests':('Testing',lambda x: x>1e5),
              'General curfew':('Asymptomatic isolation - blanket',lambda x: (x>=1) and (x<2)),
              'General strict isolation':('Asymptomatic isolation - blanket',lambda x: x>=2)}

for rc, (vc, f) in REPORTCOLS.items():
    data[rc] = data[vc].apply(f).astype(int)
    
outdata = data[list(REPORTCOLS.keys())+['Country','Date']]

outdata.to_csv(f'interventions_visualisation_boolean_{DATE}.csv')



In [4]:
def gatherings(v):
    [n] = v
    if n < 1:
        return 0
    elif n <= 10:
        return 10
    elif n <= 100:
        return 100
    elif n <= 500:
        return 500
    return 0

def tests(v):
    [n] = v
    if n < 1:
        return 0

    return math.pow(10, round(math.log(n, 10)))

def curfew(v):
    [n] = v
    if n < 1:
        return 'none'
    elif n < 2:
        return 'general'
    else:
        return 'strict'

def businesses(n):
    [a,b] = n
    if b >= 1:
        return 'many'
    elif a >= 1:
        return 'some'
    else:
        return 'none'

def social(v):
    [n] = v
    if n >= 1:
        return 'distancing'
    elif n >= 3:
        return 'stay-at-home'
    else:
        return 'none'

def isolation(n):
    [a, s] = n
    if a >= 1:
        return 'contacts'
    elif s >= 1:
        return 'cases'
    else:
        return 'none'

REPORTCOLS1 = {'mask-wearing':(['Mask wearing'],lambda x: x.head() / 100),
              'isolation':(['Symptomatic isolation - targeted','Asymptomatic isolation - targeted'],isolation),
              'gatherings':(['Gatherings banned'], gatherings),
              'businesses':(['Activity cancellation', 'Nonessential business suspension'], businesses ),
              'schools':(['School closure'], lambda x: x.head()),
              'social':(['Public interaction and hygiene'], social),
              'testing':(['Testing'], tests),
              'curfew':(['Asymptomatic isolation - blanket'],curfew)}

In [5]:


data0 = data.copy().set_index("Country")

for rc, (vc, f) in REPORTCOLS1.items():
    data0[rc] = data0[vc].apply(f, axis=1)
    
outdata = data0[list(REPORTCOLS1.keys())+['Date']]
outdata.to_csv(f'interventions_visualisation_semiquantified_{DATE}_.csv')